# Project 3: Making an AI agent with LLAMAINDEX - FunctionTool

### Follow the instructions 
Here are [instructions](https://huggingface.co/learn/agents-course/unit2/llama-index/tools) for the tutorial that are helpful for this notebook.


### Description:
This project uses `llamaindex`, a library that provides a framework for developing your agents with ease.

We look at Tools here. 

`Tools` are the fundamental objects used to build the agents.



### For this course, I am using 


see also [github code](https://github.com/huggingface/agents-course)



## Load Imports

In [1]:
import os
from pathlib import Path

import chromadb
from datasets import load_dataset
from dotenv import load_dotenv
from huggingface_hub import login

from llama_index.core.tools import FunctionTool

from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.tools.google import GmailToolSpec
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent, ToolCallResult, ToolCall
from llama_index.core.workflow import Context
from llama_index.vector_stores.chroma import ChromaVectorStore

##### Login to the Hugging Face Hub to Have Access to the Serveless Inference API

Note: You will need to add your token when prompted.


`HF_TOKEN_INFERENCE2`

In [2]:
# Load environment variables from .env
load_dotenv()

True

In [3]:
# Get Hugging Face Token
HF_TOKEN_INFERENCE2 = os.environ.get("HF_TOKEN_INFERENCE2")

----

# Creating a FunctionalTool

A FunctionTool provides a simple way to wrap any Python function and make it available to an agent. You can pass either a synchronous or asynchronous function to the tool, along with optional name and `description` parameters. The name and description are particularly important as they help the agent understand when and how to use the tool effectively. Let’s look at how to create a FunctionTool below and then call it.

Awesome, now we have a local directory with all the personas that will be attending the party, we can load and index!

In [4]:
def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny"

tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location.",
)
tool.call("New York")

Getting weather for New York


ToolOutput(blocks=[TextBlock(block_type='text', text='The weather in New York is sunny')], tool_name='my_weather_tool', raw_input={'args': ('New York',), 'kwargs': {}}, raw_output='The weather in New York is sunny', is_error=False)

---

## Creating a QueryEngineTool

The QueryEngine we defined in the previous unit can be easily transformed into a tool using the QueryEngineTool class. Let’s see how to create a QueryEngineTool from a QueryEngine in the example below.

In [5]:
embed_model = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
query_engine = index.as_query_engine(llm=llm)
tool = QueryEngineTool.from_defaults(query_engine, name="some useful name", description="some useful description")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [6]:
tool

---
## Creating Toolspecs

Think of ToolSpecs as collections of tools that work together harmoniously - like a well-organized professional toolkit. Just as a mechanic’s toolkit contains complementary tools that work together for vehicle repairs, a ToolSpec combines related tools for specific purposes. For example, an accounting agent’s ToolSpec might elegantly integrate spreadsheet capabilities, email functionality, and calculation tools to handle financial tasks with precision and efficiency.

Install the Google Toolspec
And now we can load the toolspec and convert it to a list of tools.

In [7]:
tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()

In [8]:
tool_spec_list

To get a more detailed view of the tools, we can take a look at the metadata of each tool.

In [9]:
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\n\n        Searches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query (str): The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n\n        "),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\n\n        Create and insert a draft email.\n           Print the returned draft's message and id.\n           Returns: Draft object, including draft id and message meta data.\n\n        Args:\n            to (Optional[str]): The email addresses to send the message to\n     

## Model Context Protocol `(MCP)` in LlamaIndex

LlamaIndex also allows using MCP tools through a [ToolSpec on the LlamaHub](https://llamahub.ai/l/tools/llama-index-tools-mcp?from=). You can simply run an MCP server and start using it through the following implementation.


Install the MCP Toolspec